In [2]:
!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()
findspark.init()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=6bd580430680ca9dc9eea2426530438532978271a55e967c032ca45e475a9cd1
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
homesales_df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep = ",", header= True)
homesales_df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
homesales_df.createOrReplaceTempView('home_sales')


In [10]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql('SELECT year(date) AS year, ROUND(AVG(price), 2)AS average_price FROM home_sales WHERE bedrooms = 4 GROUP BY year(date) Order By year(date) ASC').show()


+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [11]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql('SELECT year(date) AS year, ROUND(AVG(price), 2)AS average_price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY year(date) Order By year(date) ASC').show()


+----+-------------+
|year|average_price|
+----+-------------+
|2019|    287287.82|
|2020|    294204.16|
|2021|    294211.46|
|2022|    292725.69|
+----+-------------+



In [12]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql('SELECT year(date) AS year, ROUND(AVG(price), 2)AS average_price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 AND floors =2 AND sqft_living >=2000 GROUP BY year(date) Order By year(date) ASC').show()


+----+-------------+
|year|average_price|
+----+-------------+
|2019|    289859.14|
|2020|    292289.09|
|2021|    296330.96|
|2022|    290242.99|
+----+-------------+



In [14]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """
SELECT view, round(avg(price),2) AS Average_Price
FROM home_sales
WHERE price >=350000
GROUP BY view
ORDER BY round(avg(price),2) ASC
"""

spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|Average_Price|
+----+-------------+
|  33|    393973.32|
|  37|    394964.36|
|   6|    395655.38|
|  42|     396964.5|
|   2|    397389.25|
|  29|    397771.65|
|  30|     397862.0|
|  38|    398418.37|
|  47|     398447.5|
|  17|    398474.49|
|  14|    398570.03|
|   8|    398592.71|
|   3|     398867.6|
|  13|    398917.98|
|  19|    398953.17|
|  39|    399234.22|
|  18|    399332.91|
|  32|    399367.72|
|  20|    399522.81|
|  27|    399537.66|
+----+-------------+
only showing top 20 rows

--- 1.985109567642212 seconds ---


In [15]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [16]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [17]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query = """
SELECT view, round(avg(price),2) AS Average_Price
FROM home_sales
WHERE price >=350000
GROUP BY view
ORDER BY round(avg(price),2) ASC
"""

spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|Average_Price|
+----+-------------+
|  33|    393973.32|
|  37|    394964.36|
|   6|    395655.38|
|  42|     396964.5|
|   2|    397389.25|
|  29|    397771.65|
|  30|     397862.0|
|  38|    398418.37|
|  47|     398447.5|
|  17|    398474.49|
|  14|    398570.03|
|   8|    398592.71|
|   3|     398867.6|
|  13|    398917.98|
|  19|    398953.17|
|  39|    399234.22|
|  18|    399332.91|
|  32|    399367.72|
|  20|    399522.81|
|  27|    399537.66|
+----+-------------+
only showing top 20 rows

--- 0.6253583431243896 seconds ---


In [20]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
homesales_df.write.parquet('parquet_sales', mode = 'overwrite')
homesales_df.write.partitionBy('date_built').mode('overwrite').parquet('parquet_sales')

In [21]:
# 11. Read the parquet formatted data.
p_df_p = spark.read.parquet('parquet_sales')

In [22]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_home_sales_p')

In [23]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query = """
SELECT view, round(avg(price),2) AS Average_Price
FROM p_home_sales_p
WHERE price >=350000
GROUP BY view
ORDER BY round(avg(price),2) ASC
"""

spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|Average_Price|
+----+-------------+
|  33|    393973.32|
|  37|    394964.36|
|   6|    395655.38|
|  42|     396964.5|
|   2|    397389.25|
|  29|    397771.65|
|  30|     397862.0|
|  38|    398418.37|
|  47|     398447.5|
|  17|    398474.49|
|  14|    398570.03|
|   8|    398592.71|
|   3|     398867.6|
|  13|    398917.98|
|  19|    398953.17|
|  39|    399234.22|
|  18|    399332.91|
|  32|    399367.72|
|  20|    399522.81|
|  27|    399537.66|
+----+-------------+
only showing top 20 rows

--- 1.0102953910827637 seconds ---


In [24]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [25]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False